In [5]:
import pandas as pd
from io import StringIO

# CSV data as a string
csv_data = """
Rounds,Red,Blue,Game 1 Time,Game 1 Red Champ,Game 1 Blue Champ,Game 1 Red DMG,Game 1 Blue DMG,Game 1 Red CS,Game 1 Blue CS,Game 1 Winner,Game 2 Time,Game 2 Red Champ,Game 2 Blue Champ,Game 2 Red DMG,Game 2 Blue DMG,Game 2 Red CS,Game 2 Blue CS,Game 2 Winner,Game 3 Indicator,Game 3 Time,Game 3 Red Champ,Game 3 Blue Champ,Game 3 Red DMG,Game 3 Blue DMG,Game 3 Red CS,Game 3 Blue CS,Game 3 Winner,Game Winner
1,Zuoyuan,Domoya,3:03,Fiora,Shen,1612,1147,4,5,Zuoyuan,4:00,Aurora,Chogath,1726,761,14,11,Zuoyuan,0,,,,,,,,,Zuoyuan
2,Domoya,JC,2:45,Shen,Camile,963,901,3,9,Domoya,8:07,Aatrox,Ksante,3642,2061,53,44,Domoya,0,,,,,,,,,Domoya
3,Domoya,Viper3,5:10,Tahm Kech,Aatrox,1133,2087,24,29,Viper3,5:40,Sion,Fiora,2819,1417,27,26,Domoya,1,9:21,Nasus,Sett,3681,3880,40,74,Viper3,Viper3
4,Zuoyuan,Viper3,4:03,Yone,Fiora,851,1372,7,24,Viper3,3:33,Renekton,Jayce,757,757,15,14,Zuoyuan,1,3:28,Jax,Aatrox,1117,846,13,5,Zuoyuan,Zuoyuan
"""

# Load the CSV data into a DataFrame
data = pd.read_csv(StringIO(csv_data))

In [6]:
import sys
print(sys.executable)

/Users/tigerweng/anaconda3/bin/python


In [7]:
!pip3 install ace_tools

In [8]:
# Extract all games into one DataFrame
games = []
for i in range(1, 4):
    game_cols = [
        f"Game {i} Red Champ",
        f"Game {i} Blue Champ",
        f"Game {i} Winner",
    ]
    if f"Game {i} Winner" in data.columns:
        game_data = data.loc[data[f"Game {i} Winner"].notna(), ["Rounds", "Red", "Blue"] + game_cols]
        game_data = game_data.rename(columns={
            "Red": "Red Player",
            "Blue": "Blue Player",
            f"Game {i} Winner": "Winner",
        })
        games.append(game_data)

# Combine all game data into a single DataFrame
games_df = pd.concat(games, ignore_index=True)

# Calculate stats for each player
players = set(games_df["Red Player"]).union(games_df["Blue Player"])
stats = {player: {"Games Played": 0, "Wins": 0, "Loses": 0} for player in players}

for _, row in games_df.iterrows():
    red_player = row["Red Player"]
    blue_player = row["Blue Player"]
    winner = row["Winner"]
    
    stats[red_player]["Games Played"] += 1
    stats[blue_player]["Games Played"] += 1
    
    if winner == red_player:
        stats[red_player]["Wins"] += 1
        stats[blue_player]["Loses"] += 1
    elif winner == blue_player:
        stats[blue_player]["Wins"] += 1
        stats[red_player]["Loses"] += 1

# Calculate derived metrics and rank by score
stats_df = pd.DataFrame.from_dict(stats, orient="index")
stats_df["Net Wins"] = stats_df["Wins"] - stats_df["Loses"]
stats_df["Score"] = stats_df["Wins"] * 3
stats_df["Rank"] = stats_df["Score"].rank(ascending=False, method="min").astype(int)
stats_df = stats_df.sort_values(by="Score", ascending=False)
